In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv', index_col = 'Id')
test = pd.read_csv('./data/test.csv', index_col = 'Id')

In [44]:
df1 = train.copy()
df2 = test.copy()

In [19]:
df1.columns
df1.sample(5)
pd.set_option('display.max_rows', 81)
pd.set_option('display.max_columns', 999)

In [45]:
pd.DataFrame({'isna':df1.isnull().sum(), 'type':df1.dtypes})

,isna,type
MSSubClass,0,int64
MSZoning,0,object
LotFrontage,259,float64
LotArea,0,int64
Street,0,object
Alley,1369,object
LotShape,0,object
LandContour,0,object
Utilities,0,object
LotConfig,0,object


In [49]:
# Hyelee
## Alley : convert NaN => NA
df1['Alley'].fillna('NA',inplace=True)
## LotFrontage : convert NaN => ???? 
# <CASE1> Using LotArea 
df1['LotFrontage'].fillna(df1['LotArea'] *0.007206024910841549,inplace=True)

# Alyssa
# fill in for masonry stuff with None and 0
df1['MasVnrType'].fillna('None',inplace=True)
df1['MasVnrArea'].fillna('None',inplace=True)
df1.loc[949,'BsmtExposure'] = 'No' # impute the ID949's BsmtExposure with the mode of 'BsmtExposure'

# Kisoo
df1.FireplaceQu = df1.FireplaceQu.fillna('NA') # without Fireplace, there is no FireplaceQu.
df1.Electrical = df1.Electrical.fillna('SBrkr') # since Utility column, there is electricity obviously, so it filled with most common Electrical type 'SBrkr'

# Wonchan
# Feature Engineering for Time Series Columns
df1['GarageYrBltImputed'] = np.where(df1['GarageYrBlt'].isnull(), 1, 0)

# impute the missing years with the value of the year built plus the mean of the diff of year built and garageyrbuilt
aveDiff = np.mean(df1['GarageYrBlt']-df1['YearBuilt'])
df1['GarageYrBlt'].fillna(df1['GarageYrBlt'] + aveDiff,inplace=True)

# fill rest with NA string
df1 = df1.fillna('NA')


In [ ]:
# Hyelee
## Alley : convert NaN => NA
df1['Alley'].fillna('NA',inplace=True)
## LotFrontage : convert NaN => ???? 
# <CASE2> Using Neighborhood 
df_tmp = df1[~df1.LotFrontage.isnull()]
df_md_lotfrontage = df_tmp.groupby('Neighborhood').agg('median')[{'LotFrontage'}]
df_md_lotfrontage = df_md_lotfrontage.rename(columns={"LotFrontage" : "mdLotFrontage"})
df_tmp = df_tmp.merge(df_md_lotfrontage, how="inner", on='Neighborhood')
df1= pd.concat([df1.reset_index().set_index('Neighborhood'),df_md_lotfrontage], axis=1, join='inner').reset_index().set_index('Id')
df1['LotFrontage'].fillna(df1['mdLotFrontage'],inplace=True)
df1= df1.drop(columns=['mdLotFrontage'])

# Alyssa
# fill in for masonry stuff with None and 0
df1['MasVnrType'].fillna('None',inplace=True)
df1['MasVnrArea'].fillna('None',inplace=True)
df1.loc[949,'BsmtExposure'] = 'No' # impute the ID949's BsmtExposure with the mode of 'BsmtExposure'

# Kisoo
df1.FireplaceQu = df1.FireplaceQu.fillna('NA') # without Fireplace, there is no FireplaceQu.
df1.Electrical = df1.Electrical.fillna('SBrkr') # since Utility column, there is electricity obviously, so it filled with most common Electrical type 'SBrkr'

# Wonchan
# Feature Engineering for Time Series Columns
df1['GarageYrBltImputed'] = np.where(df1['GarageYrBlt'].isnull(), 1, 0)

# impute the missing years with the value of the year built plus the mean of the diff of year built and garageyrbuilt
aveDiff = np.mean(df1['GarageYrBlt']-df1['YearBuilt'])
df1['GarageYrBlt'].fillna(df1['GarageYrBlt'] + aveDiff,inplace=True)

# fill rest with NA string
df1 = df1.fillna('NA')